En este notebook traté de mejorar el modelo NB para inglés tras pipeline, pero algo va mal porque han aparecido NAN que no deberian existir.

In [3]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk

# Descargar el paquete 'punkt' para el tokenizador
nltk.download('punkt')

# Función de stemming
stemmer = SnowballStemmer("english")
def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in word_tokenize(text)])

# Cargar el dataset modificado
df_modificado = pd.read_csv('../data_en/spam_ham_dataset_modificado.csv')

# Aplicar stemming
df_modificado['text'] = df_modificado['text'].apply(stem_text)

# Dividir el dataset en conjunto de entrenamiento, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(df_modificado['text'], df_modificado['label_num'], test_size=0.2, random_state=42, stratify=df_modificado['label_num'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Crear y entrenar el pipeline con bigramas y stemming, usando SMOTE
pipeline = ImbPipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
    ('smote', SMOTE(random_state=42)),
    ('classifier', MultinomialNB())
])
pipeline.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de validación
y_val_pred = pipeline.predict(X_val)
print("Reporte de Clasificación en el conjunto de validación:")
print(classification_report(y_val, y_val_pred, target_names=['ham', 'spam']))
print("Matriz de Confusión en el conjunto de validación:")
print(confusion_matrix(y_val, y_val_pred))
print("Exactitud en el conjunto de validación:")
print(accuracy_score(y_val, y_val_pred))

# Guardar el modelo y el vectorizador
joblib.dump(pipeline, '../modelos_y_vectorizadores_en/naive_bayes_smote_pipeline_improved.pkl')

# Evaluar el modelo en el conjunto de prueba
y_test_pred = pipeline.predict(X_test)
print("Reporte de Clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_test_pred, target_names=['ham', 'spam']))
print("Matriz de Confusión en el conjunto de prueba:")
print(confusion_matrix(y_test, y_test_pred))
print("Exactitud en el conjunto de prueba:")
print(accuracy_score(y_test, y_test_pred))

# Identificar los casos donde el modelo se ha equivocado
label_mapping = {0: 'ham', 1: 'spam'}
y_test_mapped = y_test.map(label_mapping)
y_test_pred_mapped = pd.Series(y_test_pred).map(label_mapping)
results_df = pd.DataFrame({'text': X_test, 'actual': y_test_mapped, 'predicted': y_test_pred_mapped})
incorrect_cases = results_df[results_df['actual'] != results_df['predicted']]

# Mostrar los casos incorrectos
print("Casos donde el modelo se ha equivocado:")
print(incorrect_cases)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ferna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Reporte de Clasificación en el conjunto de validación:
              precision    recall  f1-score   support

         ham       1.00      0.96      0.98       707
        spam       0.91      0.99      0.95       292

    accuracy                           0.97       999
   macro avg       0.95      0.98      0.96       999
weighted avg       0.97      0.97      0.97       999

Matriz de Confusión en el conjunto de validación:
[[679  28]
 [  2 290]]
Exactitud en el conjunto de validación:
0.96996996996997
Reporte de Clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

         ham       1.00      0.95      0.97       706
        spam       0.88      0.99      0.93       293

    accuracy                           0.96       999
   macro avg       0.94      0.97      0.95       999
weighted avg       0.96      0.96      0.96       999

Matriz de Confusión en el conjunto de prueba:
[[668  38]
 [  3 290]]
Exactitud en el conjunto de prueba:
0.958

Algo no ha ido bien porque muestra NaN que antes no habia

In [4]:
df_modificado.describe()

,Unnamed: 0,label_num
count,4993.000000,4993.000000
mean,2581.528139,0.292810
std,1505.740921,0.455098
min,0.000000,0.000000
25%,1251.000000,0.000000
50%,2578.000000,0.000000
75%,3886.000000,1.000000
max,5170.000000,1.000000


In [5]:
df_modificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4993 non-null   int64 
 1   label       4993 non-null   object
 2   text        4993 non-null   object
 3   label_num   4993 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 156.2+ KB
